In [1]:
import numpy as np
from SERGIO.GRN import grn_from_human,grn_from_networkx
from SERGIO._sergio import sergio
from SERGIO.MR import mrProfile
import pandas as pd
from SERGIO.GRN._grn import GRN
from SERGIO.GRN._components import Gene, SingleInteraction


Use regulatory network from Zhi-Ping Liu, Canglin Wu, Hongyu Miao and Hulin Wu (2015). RegNetwork: an integrated database of transcriptional and posttranscriptional regulatory networks in human and mouse. Database 2015. doi: [10.1093/database/bav095](10.1093/database/bav095)

In [2]:
grn = grn_from_human()

In [3]:

mrs = grn.get_mrs()
mr_profs = mrProfile(MR_names = mrs, n_types = 1)
mr_profs.build_rnd(range_dict={'L': [1, 2.5], 'H': [3.5, 5]})


In [4]:
grn.init(mr_profs, update_half_resp = True)
sim = sergio(grn)
%time sim.simulate(nCells = 100, noise_s = 1, safety_iter = 150, scale_iter = 10)

CPU times: user 12.5 s, sys: 106 µs, total: 12.5 s
Wall time: 12.5 s


In [8]:
expr = sim.getSimExpr()

In [6]:
grn.init(mr_profs, update_half_resp = True)
sim_gpu = sergio(grn)
%time sim_gpu.simulate_gpu(nCells = 1000, noise_s = 1, safety_iter = 150, scale_iter = 10)

CPU times: user 8min 58s, sys: 158 ms, total: 8min 58s
Wall time: 8min 58s


# Use custom graph from networkx

In [2]:
import networkx as nx

In [11]:
# create a networkx.DiGraph()
G = nx.DiGraph()
G.add_edges_from([('1','2',{'weight':2}),('2','3',{'weight':-1}),('3','1',{'weight':3})])# if weight is not given, grn_from_networkx will parametrize automatically

# convert the networkx.DiGraph() to a GRN object
grn = grn_from_networkx(g= G, parametrize=False)

In [12]:
mrs = grn.get_mrs()
mr_profs = mrProfile(MR_names = mrs, n_types = 2)
mr_profs.build_rnd(range_dict={'L': [1, 2.5], 'H': [3.5, 5]})
grn.init(mr_profs, update_half_resp = True)
sim = sergio(grn)
sim.simulate(nCells = 200, noise_s = 1, safety_iter = 150, scale_iter = 10)

In [13]:
A = grn.to_networkx()

In [23]:
a = set(A.edges())
a.add(1)#note that Sergio removes cycles

## Add Technical Noise _ Steady-State Simulations

In [12]:
"""
Add outlier genes
"""
expr_O = sim.outlier_effect(expr, outlier_prob = 0.01, mean = 0.8, scale = 1)

"""
Add Library Size Effect
"""
libFactor, expr_O_L = sim.lib_size_effect(expr_O, mean = 4.6, scale = 0.4)

"""
Add Dropouts
"""
binary_ind = sim.dropout_indicator(expr_O_L, shape = 6.5, percentile = 82)
expr_O_L_D = np.multiply(binary_ind, expr_O_L)

"""
Convert to UMI count
"""
count_matrix = sim.convert_to_UMIcounts(expr_O_L_D)

"""
Make a 2d gene expression matrix
"""
count_matrix = np.concatenate(count_matrix, axis = 1)

KeyError: 0

In [14]:
len(expr)

405

In [ ]:
def outlier_effect(scData, outlier_prob, mean, scale):
    """
    This function
    """
    out_indicator = np.random.binomial(n = 1, p = outlier_prob, size = len(scData))
    outlierGenesIndx = np.where(out_indicator == 1)[0]
    numOutliers = len(outlierGenesIndx)

    #### generate outlier factors ####
    outFactors = np.random.lognormal(mean = mean, sigma = scale, size = numOutliers)
    ##################################

    scData = np.concatenate(scData, axis = 1)
    for i, gIndx in enumerate(outlierGenesIndx):
        scData[gIndx,:] = scData[gIndx,:] * outFactors[i]

    return np.split(scData, self.nBins_, axis = 1)
